In [1]:
import numpy as np
import os
import time

In [2]:
import keras

In [3]:
keras.__version__

'2.4.3'

In [4]:
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [5]:
#!pip install -q -U tensorflow-addons==0.11.2

### Utilities

In [6]:
from utils import verifyDir
from utils.networks import normalize, unnormalize, plot_data

### Dataset

In [7]:
from utils.CIFAR10 import load_real_samples

### Discriminator & Generator

In [8]:
from utils.CIFAR10 import define_discriminator
from utils.CIFAR10 import define_generator

### Semi-Supervised GAN

In [9]:
from utils.networks import define_gan

### Selecting sub-set 

In [10]:
from utils.networks import select_supervised_samples, generate_real_samples
from utils.networks import generate_fake_samples, generate_latent_points

### Training

In [11]:
# train the generator and discriminator
def train(generator_model, unsupervised_model, supervised_model, gan_model, dataset_train, dataset_test, 
          latent_dim=100, n_epochs=20, n_batch=100, percent_samples=1.0, n_classes=10):
    
    # select supervised dataset_train
    X_sup, y_sup = select_supervised_samples(dataset_train, percent_samples=percent_samples, n_classes=n_classes)
    print("Sup samples:", X_sup.shape, y_sup.shape)
    
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset_train[0].shape[0] / n_batch)
    
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    print('n_epochs=%d, n_batch=%d, batch/epoch=%d, steps=%d' % (n_epochs, n_batch, bat_per_epo, n_steps))
    
    # manually enumerate epochs
    f_history = open(f"{LOG_PATH}SSL_GAN.csv", "w")
    f_history.write("step,generator_loss,unsupervised_real_loss,unsupervised_fake_loss,supervised_loss,supervised_acc,train_loss,test_loss,train_acc,test_acc\n")
    
    #for epoch in n_epochs:
    #    for batch in range(bat_per_epo):
    for step in range(1,n_steps+1):
#         t_start = time.time()
        # update supervised discriminator (c)
        [Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], n_batch)
        c_loss, c_acc = supervised_model.train_on_batch(Xsup_real, ysup_real)
        
        # update unsupervised discriminator (d)
        [X_real, _], y_real = generate_real_samples(dataset_train, n_batch)
        d_loss1, real_acc = unsupervised_model.train_on_batch(X_real, y_real)
        
        X_fake, y_fake = generate_fake_samples(generator_model, latent_dim, n_batch)
        d_loss2, fake_acc = unsupervised_model.train_on_batch(X_fake, y_fake)
        
        # update generator (g)
        X_gan, y_gan = generate_latent_points(latent_dim, n_batch), np.ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
#         t_total = (time.time() - t_start)
        # summarize loss on this batch
    
        # Train - Test
        X_train, y_train = dataset_train
        loss_train, acc_train = supervised_model.evaluate(X_train, y_train, verbose=0)

        # evaluate the test classifier model
        X_test, y_test = dataset_test
        loss_test, acc_test = supervised_model.evaluate(X_test, y_test, verbose=0)
        
        # Log
        print('epoch: %d | step: %d | Train: G_Loss: %.3f, ' \
              'D_unsup_loss_real: %.3f, D_unsup_acc_real:  %.2f, ' \
              'D_unsup_loss_fake: %.3f, D_unsup_acc_fake: %.2f, ' \
              'D_sup_loss: %.3f, D_sup_acc: %.2f ' \
              'Train acc: %.3f Test acc: %.3f ' %(int(step/bat_per_epo), step, g_loss,
                                                d_loss1, real_acc*100,
                                                d_loss2, fake_acc*100,
                                                c_loss, c_acc*100,
                                                 acc_train*100, acc_test*100))#, end = '\r')
        f_history.write(f"{step},{g_loss},{d_loss1},{d_loss2},{c_loss},{c_acc*100},{loss_train},{loss_test},{acc_train*100},{acc_test*100}\n")
        
        if step==1:
            plot_data(unnormalize(X_test).astype(int), 0, "test", grid_size = [10, 10], OUT_PATH=LOG_PATH)
        # evaluate the model performance every so often
        if (step) % (100) == 0 or step == 1:
            #summarize_performance(step, generator_model, supervised_model, latent_dim, dataset, dataset_test)
            # prepare fake examples
            X_generated, _ = generate_fake_samples(generator_model, latent_dim, n_samples=100)
            # scale from [-1,1] to [0,255]
            plot_data(unnormalize(X_generated).astype(int), step, "generated", grid_size = [10, 10], OUT_PATH=LOG_PATH)
            
            X_train, y_train = dataset_train
            _, acc = supervised_model.evaluate(X_train, y_train, verbose=0)
            print('Train Classifier Accuracy: %.3f%%\n' % (acc * 100))
            
            # evaluate the test classifier model
            X_test, y_test = dataset_test
            _, acc = supervised_model.evaluate(X_test, y_test, verbose=0)
            print('Test Classifier Accuracy: %.3f%%\n' % (acc * 100))
            # save the generator model
            filename2 = f'{LOG_PATH}generator_model_{step}.h5'
            generator_model.save(filename2)
            # save the classifier model
            filename3 = f'{LOG_PATH}supervised_model_{step}.h5'
            supervised_model.save(filename3)
            print('>Saving models Generator: %s and Supervised: %s' % (filename2, filename3))
    
    f_history.close()

### Parameters

In [12]:
input_shape = (32, 32, 3)
num_classes = 10

learning_rate =  0.0002
latent_dim = 100

labeled_rate = 1.0
labeled_samples = 50000*labeled_rate

In [13]:
LOG_PATH = f"Logs/SSGAN_CIFAR10/Classifier_{int(labeled_samples)}/"
verifyDir(LOG_PATH)

### Creating models

In [14]:
# create the discriminator models
unsupervised_model, supervised_model = define_discriminator(in_shape=input_shape, n_classes=num_classes, learning_rate = learning_rate)
# create the generator
generator_model = define_generator(latent_dim=latent_dim)

In [15]:
supervised_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        36928 

In [16]:
unsupervised_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        3692

In [17]:
generator_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              413696    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 4096)              0         
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 128)         524416    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 16, 16, 128)       2622

In [18]:
# create the gan
gan_model = define_gan(generator_model, unsupervised_model, learning_rate = learning_rate)

In [19]:
gan_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              413696    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 4096)              0         
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 128)         524416    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 16, 16, 128)       2622

### Loading Dataset

In [20]:
# load image data
dataset_train, dataset_test = load_real_samples()

(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)


### Training

In [21]:
epochs=300
batch_size=128

In [ ]:
train(generator_model, unsupervised_model, supervised_model, gan_model, 
      dataset_train, dataset_test, latent_dim=latent_dim, 
      n_epochs=epochs, n_batch=batch_size, percent_samples=labeled_rate, n_classes=num_classes)

Sup samples: (50000, 32, 32, 3) (50000,)
n_epochs=300, n_batch=128, batch/epoch=390, steps=117000
epoch: 0 | step: 1 | Train: G_Loss: 0.095, D_unsup_loss_real: 0.095, D_unsup_acc_real:  100.00, D_unsup_loss_fake: 2.401, D_unsup_acc_fake: 0.00, D_sup_loss: 2.310, D_sup_acc: 5.47 Train acc: 13.662 Test acc: 13.800 
Train Classifier Accuracy: 13.662%

Test Classifier Accuracy: 13.800%

>Saving models Generator: Logs/SSGAN_CIFAR10/Classifier_50000/generator_model_1.h5 and Supervised: Logs/SSGAN_CIFAR10/Classifier_50000/supervised_model_1.h5
epoch: 0 | step: 2 | Train: G_Loss: 0.096, D_unsup_loss_real: 0.089, D_unsup_acc_real:  100.00, D_unsup_loss_fake: 2.398, D_unsup_acc_fake: 0.00, D_sup_loss: 2.299, D_sup_acc: 13.81 Train acc: 15.378 Test acc: 15.230 
epoch: 0 | step: 3 | Train: G_Loss: 0.096, D_unsup_loss_real: 0.083, D_unsup_acc_real:  100.00, D_unsup_loss_fake: 2.395, D_unsup_acc_fake: 0.00, D_sup_loss: 2.296, D_sup_acc: 15.16 Train acc: 16.476 Test acc: 16.660 
epoch: 0 | step: 4 | 

epoch: 0 | step: 37 | Train: G_Loss: 2.741, D_unsup_loss_real: 0.454, D_unsup_acc_real:  78.91, D_unsup_loss_fake: 0.420, D_unsup_acc_fake: 92.19, D_sup_loss: 2.442, D_sup_acc: 21.90 Train acc: 21.758 Test acc: 22.340 
epoch: 0 | step: 38 | Train: G_Loss: 2.899, D_unsup_loss_real: 0.477, D_unsup_acc_real:  78.91, D_unsup_loss_fake: 0.375, D_unsup_acc_fake: 95.31, D_sup_loss: 2.465, D_sup_acc: 22.30 Train acc: 21.946 Test acc: 22.550 
epoch: 0 | step: 39 | Train: G_Loss: 2.907, D_unsup_loss_real: 0.479, D_unsup_acc_real:  76.56, D_unsup_loss_fake: 0.254, D_unsup_acc_fake: 97.66, D_sup_loss: 2.607, D_sup_acc: 22.43 Train acc: 22.202 Test acc: 22.820 
epoch: 0 | step: 40 | Train: G_Loss: 2.742, D_unsup_loss_real: 0.446, D_unsup_acc_real:  78.12, D_unsup_loss_fake: 0.218, D_unsup_acc_fake: 99.22, D_sup_loss: 2.467, D_sup_acc: 22.74 Train acc: 22.704 Test acc: 23.580 
epoch: 0 | step: 41 | Train: G_Loss: 3.143, D_unsup_loss_real: 0.292, D_unsup_acc_real:  85.16, D_unsup_loss_fake: 0.210, D_

epoch: 0 | step: 75 | Train: G_Loss: 2.004, D_unsup_loss_real: 0.409, D_unsup_acc_real:  78.91, D_unsup_loss_fake: 0.274, D_unsup_acc_fake: 95.31, D_sup_loss: 2.198, D_sup_acc: 24.50 Train acc: 22.692 Test acc: 23.530 
epoch: 0 | step: 76 | Train: G_Loss: 1.991, D_unsup_loss_real: 0.461, D_unsup_acc_real:  72.66, D_unsup_loss_fake: 0.318, D_unsup_acc_fake: 92.19, D_sup_loss: 2.210, D_sup_acc: 23.45 Train acc: 22.804 Test acc: 23.410 
epoch: 0 | step: 77 | Train: G_Loss: 1.810, D_unsup_loss_real: 0.586, D_unsup_acc_real:  71.88, D_unsup_loss_fake: 0.386, D_unsup_acc_fake: 91.41, D_sup_loss: 2.236, D_sup_acc: 23.33 Train acc: 23.310 Test acc: 23.890 
epoch: 0 | step: 78 | Train: G_Loss: 2.224, D_unsup_loss_real: 0.295, D_unsup_acc_real:  85.16, D_unsup_loss_fake: 0.244, D_unsup_acc_fake: 99.22, D_sup_loss: 2.248, D_sup_acc: 23.82 Train acc: 22.776 Test acc: 23.520 
epoch: 0 | step: 79 | Train: G_Loss: 1.693, D_unsup_loss_real: 0.592, D_unsup_acc_real:  66.41, D_unsup_loss_fake: 0.353, D_

epoch: 0 | step: 112 | Train: G_Loss: 1.830, D_unsup_loss_real: 0.420, D_unsup_acc_real:  74.22, D_unsup_loss_fake: 0.294, D_unsup_acc_fake: 96.88, D_sup_loss: 2.078, D_sup_acc: 26.92 Train acc: 28.046 Test acc: 27.540 
epoch: 0 | step: 113 | Train: G_Loss: 1.844, D_unsup_loss_real: 0.482, D_unsup_acc_real:  75.00, D_unsup_loss_fake: 0.465, D_unsup_acc_fake: 85.16, D_sup_loss: 2.092, D_sup_acc: 27.43 Train acc: 27.430 Test acc: 27.120 
epoch: 0 | step: 114 | Train: G_Loss: 1.879, D_unsup_loss_real: 0.344, D_unsup_acc_real:  82.03, D_unsup_loss_fake: 0.411, D_unsup_acc_fake: 85.16, D_sup_loss: 2.113, D_sup_acc: 27.07 Train acc: 26.982 Test acc: 26.800 
epoch: 0 | step: 115 | Train: G_Loss: 1.517, D_unsup_loss_real: 0.772, D_unsup_acc_real:  56.25, D_unsup_loss_fake: 0.459, D_unsup_acc_fake: 82.03, D_sup_loss: 2.121, D_sup_acc: 26.77 Train acc: 24.574 Test acc: 24.860 
epoch: 0 | step: 116 | Train: G_Loss: 1.617, D_unsup_loss_real: 0.525, D_unsup_acc_real:  67.97, D_unsup_loss_fake: 0.55

epoch: 0 | step: 150 | Train: G_Loss: 2.588, D_unsup_loss_real: 0.274, D_unsup_acc_real:  91.41, D_unsup_loss_fake: 0.765, D_unsup_acc_fake: 53.12, D_sup_loss: 1.992, D_sup_acc: 29.09 Train acc: 27.138 Test acc: 27.250 
epoch: 0 | step: 151 | Train: G_Loss: 2.281, D_unsup_loss_real: 0.435, D_unsup_acc_real:  82.03, D_unsup_loss_fake: 0.233, D_unsup_acc_fake: 97.66, D_sup_loss: 2.025, D_sup_acc: 27.19 Train acc: 27.284 Test acc: 27.410 
epoch: 0 | step: 152 | Train: G_Loss: 1.815, D_unsup_loss_real: 0.260, D_unsup_acc_real:  89.84, D_unsup_loss_fake: 0.323, D_unsup_acc_fake: 96.88, D_sup_loss: 2.036, D_sup_acc: 27.38 Train acc: 29.712 Test acc: 29.780 
epoch: 0 | step: 153 | Train: G_Loss: 1.930, D_unsup_loss_real: 0.250, D_unsup_acc_real:  94.53, D_unsup_loss_fake: 0.494, D_unsup_acc_fake: 82.81, D_sup_loss: 1.966, D_sup_acc: 29.71 Train acc: 31.546 Test acc: 31.910 
epoch: 0 | step: 154 | Train: G_Loss: 2.217, D_unsup_loss_real: 0.263, D_unsup_acc_real:  93.75, D_unsup_loss_fake: 0.57

epoch: 0 | step: 188 | Train: G_Loss: 2.103, D_unsup_loss_real: 0.389, D_unsup_acc_real:  75.00, D_unsup_loss_fake: 0.334, D_unsup_acc_fake: 92.19, D_sup_loss: 1.963, D_sup_acc: 30.10 Train acc: 29.132 Test acc: 29.960 
epoch: 0 | step: 189 | Train: G_Loss: 1.828, D_unsup_loss_real: 0.506, D_unsup_acc_real:  69.53, D_unsup_loss_fake: 0.495, D_unsup_acc_fake: 82.03, D_sup_loss: 1.962, D_sup_acc: 29.84 Train acc: 29.574 Test acc: 30.120 
epoch: 0 | step: 190 | Train: G_Loss: 1.968, D_unsup_loss_real: 0.453, D_unsup_acc_real:  71.88, D_unsup_loss_fake: 0.484, D_unsup_acc_fake: 83.59, D_sup_loss: 1.957, D_sup_acc: 30.09 Train acc: 29.638 Test acc: 30.100 
epoch: 0 | step: 191 | Train: G_Loss: 1.793, D_unsup_loss_real: 0.533, D_unsup_acc_real:  71.09, D_unsup_loss_fake: 0.471, D_unsup_acc_fake: 85.16, D_sup_loss: 1.951, D_sup_acc: 30.05 Train acc: 30.184 Test acc: 30.370 
epoch: 0 | step: 192 | Train: G_Loss: 1.748, D_unsup_loss_real: 0.500, D_unsup_acc_real:  76.56, D_unsup_loss_fake: 0.61

epoch: 0 | step: 225 | Train: G_Loss: 1.891, D_unsup_loss_real: 0.362, D_unsup_acc_real:  84.38, D_unsup_loss_fake: 0.617, D_unsup_acc_fake: 73.44, D_sup_loss: 1.995, D_sup_acc: 28.69 Train acc: 30.462 Test acc: 30.100 
epoch: 0 | step: 226 | Train: G_Loss: 1.786, D_unsup_loss_real: 0.439, D_unsup_acc_real:  78.91, D_unsup_loss_fake: 0.457, D_unsup_acc_fake: 87.50, D_sup_loss: 1.972, D_sup_acc: 30.02 Train acc: 32.294 Test acc: 31.970 
epoch: 0 | step: 227 | Train: G_Loss: 1.947, D_unsup_loss_real: 0.378, D_unsup_acc_real:  82.03, D_unsup_loss_fake: 0.505, D_unsup_acc_fake: 78.12, D_sup_loss: 1.943, D_sup_acc: 31.97 Train acc: 33.260 Test acc: 32.900 
epoch: 0 | step: 228 | Train: G_Loss: 1.821, D_unsup_loss_real: 0.468, D_unsup_acc_real:  75.78, D_unsup_loss_fake: 0.475, D_unsup_acc_fake: 83.59, D_sup_loss: 1.920, D_sup_acc: 32.77 Train acc: 33.588 Test acc: 33.200 
epoch: 0 | step: 229 | Train: G_Loss: 2.028, D_unsup_loss_real: 0.400, D_unsup_acc_real:  85.94, D_unsup_loss_fake: 0.52

### Testing

In [ ]:
dataset_train, dataset_test = load_real_samples()

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
last_step = int(dataset_train.shape[0]/batch_size)*epochs

In [ ]:
supervised_model = load_model(f'{LOG_PATH}supervised_model_{last_step}.h5')

In [ ]:
X_train, y_train = dataset_train
_, acc = supervised_model.evaluate(X_train, y_train, verbose=0)
print('Train Classifier Accuracy: %.3f%%\n' % (acc * 100))

In [ ]:
X_test, y_test = dataset_test
_, acc = supervised_model.evaluate(X_test, y_test, verbose=0)
print('Test Classifier Accuracy: %.3f%%\n' % (acc * 100))

### Plotting

In [ ]:
import pandas as pd

In [ ]:
results_file = pd.read_csv(f"{LOG_PATH}SSL_GAN.csv")

In [ ]:
log_file = results_file.iloc[:,1:]
log_file

In [ ]:
log_file.iloc[:, [0,1,2,3]].plot(figsize=(12,8))

In [ ]:
log_file.iloc[:, [5,6]].plot(figsize=(12,8))

In [ ]:
log_file.iloc[:, [7,8]].plot(figsize=(12,8), ylim=(0,100), yticks=range(0,110,10))